In [2]:
cd ..

/Users/vijay/vijaygkd/deep-ttt


In [438]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as k
import mlflow

In [6]:
#%load_ext autoreload
%autoreload 2

In [1617]:
from dqn.environment import *
from dqn.agent import RandomAgent, QAgent
from dqn.model import DQN
from dqn.train import Memory, QLearning, play_games_against_random, play_games_against_other_agent
from dqn.minmax import MinMaxAgent

In [1642]:
t = QLearning()

[autoreload of dqn.train failed: Traceback (most recent call last):
  File "/Users/vijay/.local/share/virtualenvs/deep-ttt-_cETEJsv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/vijay/.local/share/virtualenvs/deep-ttt-_cETEJsv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/vijay/.pyenv/versions/3.9.10/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/vijay/vijaygkd/deep-ttt/dqn/train.py", line 298
 

hey-14


In [1643]:
t.net.model.summary()

Model: "model_68"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_states (InputLayer)   [(None, 9)]               0         
                                                                 
 hidden_1 (Dense)            (None, 128)               1280      
                                                                 
 hidden_2 (Dense)            (None, 64)                8256      
                                                                 
 output (Dense)              (None, 9)                 585       
                                                                 
Total params: 10,121
Trainable params: 10,121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
with mlflow.start_run(run_name='run 14 - train with minmax', description='') as run:
    t.train(15000)

## Gradient Calculations

In [1414]:
X = np.array([[1., 0., 2.],
             [0., 1., 0.],
             [0., 0., 3.]])
Xt = tf.convert_to_tensor(X)

W = np.array([[1., 0.],
             [1., 1.],
             [0., 1.]])
Wt = tf.convert_to_tensor(W)

In [1415]:
Y_hat = np.dot(X, W)
Y_hat

array([[1., 2.],
       [1., 1.],
       [0., 3.]])

In [1417]:
Y = np.array([[1., 3.],
              [1., 3.],
              [0., 0.]])

In [1419]:
(1+4+9)/6

2.3333333333333335

In [1418]:
mse = tf.keras.losses.MeanSquaredError()
mse(Y, Y_hat).numpy()

2.3333333333333335

In [1420]:
Y-Y_hat

array([[ 0.,  1.],
       [ 0.,  2.],
       [ 0., -3.]])

In [1424]:
np.dot(X.T, Y-Y_hat) * -2/6

array([[-0.        , -0.33333333],
       [-0.        , -0.66666667],
       [-0.        ,  2.33333333]])

In [1422]:
with tf.GradientTape(persistent=True) as g:
    g.watch([Xt, Wt])
    Yh = Xt @ Wt
    L = mse(Y, Yh)

In [1423]:
g.gradient(L, [Wt, Xt])

[<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[ 0.        , -0.33333333],
        [ 0.        , -0.66666667],
        [ 0.        ,  2.33333333]])>,
 <tf.Tensor: shape=(3, 3), dtype=float64, numpy=
 array([[ 0.        , -0.33333333, -0.33333333],
        [ 0.        , -0.66666667, -0.66666667],
        [ 0.        ,  1.        ,  1.        ]])>]

In [1285]:
L

<tf.Tensor: shape=(), dtype=float64, numpy=1.3333333333333333>

In [1299]:
np.dot(X.T, (Y-Y_hat)) * -2 / 6

array([[-0.33333333, -0.        ],
       [ 0.33333333, -0.33333333],
       [-1.66666667,  2.        ]])

In [1291]:
np.dot(Y_hat-Y, W.T) * 2 / 6

array([[-0.33333333, -0.33333333,  0.        ],
       [ 0.33333333,  0.        , -0.33333333],
       [-0.33333333,  0.33333333,  0.66666667]])

In [1303]:
np.dot(X, (Y-Y_hat)) * -2 / 6  # ---> Wrong

array([[-1.        ,  1.33333333],
       [ 0.33333333, -0.33333333],
       [-1.        ,  2.        ]])

In [1426]:

mse = tf.keras.losses.MeanSquaredError()


## Play against AI

In [1608]:
g = TicTacToe()

In [1612]:
g.execute_action(7)

(-0.1, 0)

In [1613]:
m, q = MinMaxAgent.play(g.get_current_state())
print(m, q)
g.execute_action(m)
g.print_board()

3 0
- X O 

X X - 

- O - 



In [1552]:
m, q = t.agent.play(g.get_current_state())
print(m, q)
g.execute_action(m)
g.print_board()

0 2.1647792
X - - 

- - - 

X O - 



In [1554]:
g.get_current_state().reshape(3, 3)

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1., -1.,  0.]])

In [1555]:
g.current_player

-1

In [1614]:
s = play_games_against_random(t.agent, 100)
s

In [1629]:
s = play_games_against_other_agent(t.agent, RandomAgent(), 'random', 100)
s

{'random_win': 0.91, 'random_lost': 0.05, 'random_draw': 0.04}

In [1630]:
s = play_games_against_other_agent(t.agent, MinMaxAgent(), 'minmax', 100)
s

{'minmax_win': 0, 'minmax_lost': 0.44, 'minmax_draw': 0.56}

In [1636]:
s = play_games_against_other_agent(MinMaxAgent(), RandomAgent(), 'minmaxvs_random', 100)
s

{'minmaxvs_random_win': 0.84,
 'minmaxvs_random_lost': 0,
 'minmaxvs_random_draw': 0.16}

In [1637]:
s = play_games_against_other_agent(MinMaxAgent(), RandomAgent(), 'minmaxvs_random', 100)
s

{'minmaxvs_random_win': 0.87,
 'minmaxvs_random_lost': 0,
 'minmaxvs_random_draw': 0.13}

In [1644]:
s = play_games_against_other_agent( RandomAgent(), MinMaxAgent(), 'random_vs_minmax', 40)
s

{'random_vs_minmax_win': 0,
 'random_vs_minmax_lost': 0.83,
 'random_vs_minmax_draw': 0.18}

### Test gradient updates